# СЕРВИС ДЛЯ ЧТЕНИЯ КНИГ

**ОПИСАНИЕ ПРОЕКТА**

Задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**ОПИСАНИЕ ДАННЫХ**

Таблица **books**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица **authors**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица **publishers**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица **ratings**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица **reviews**

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

**ПОДКЛЮЧЕНИЕ К БАЗЕ ДАННЫХ**

In [1]:
!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Исследуем таблицы , которые есть в нашем распоряжении**

In [6]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**ЗАДАНИЕ 1**

Посчитать, сколько книг вышло после 1 января 2000 года

In [11]:
query = '''SELECT COUNT(book_id)
           FROM books 
           WHERE publication_date > '2000-01-01'
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


819 книг вышло после 1 января 2000 года.

**ЗАДАНИЕ 2**

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [12]:
query = '''SELECT  b.title,
                   COUNT(DISTINCT rev.review_id),
                   AVG(rat.rating)
           FROM books as b
           LEFT JOIN reviews as rev ON b.book_id = rev.book_id
           LEFT JOIN ratings as rat ON b.book_id = rat.book_id
           GROUP BY b.book_id
           ORDER BY AVG(rat.rating) DESC
         '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count,avg
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


**ЗАДАНИЕ 3**

Определить издательство, которое выпустило наибольшее число книг толще
50 страниц — так мы исключим из анализа брошюры;

In [13]:
query = '''SELECT publisher,
               COUNT(b.book_id) AS books_count
        FROM publishers AS p
        LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE num_pages > 50
        GROUP BY publisher
        ORDER BY books_count DESC
        LIMIT 1
         '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_count
0,Penguin Books,42


Издательство 'Penguin Books' выпустило 42 книги объемом больше 50 страниц.

**ЗАДАНИЕ 4**

Определить автора с самой высокой средней оценкой книг — учитываем
только книги с 50 и более оценками

In [14]:
query = '''SELECT a.author,
           ROUND(AVG(rat.rating),2)
        FROM authors AS a
        LEFT JOIN books AS b ON a.author_id = b.author_id
        LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
        WHERE b.book_id IN (SELECT book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50)
        GROUP BY author
        ORDER BY AVG(rat.rating) DESC
        LIMIT 1
         '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг - 	J.K. Rowling/Mary GrandPré	

**ЗАДАНИЕ 5**

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [15]:
query = '''
            WITH reviews_count_table AS 
            (SELECT username,
                   COUNT(review_id) AS reviews_count
            FROM reviews 
            WHERE username IN (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(rating_id) > 48)
            GROUP BY username)
            
            SELECT AVG(reviews_count) AS avg_reviews_count
            FROM reviews_count_table
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,avg_reviews_count
0,24.0


Среднее количество обзоров от пользователей - 24


**ВЫВОД**

- 819 книг вышло после 1 января 2000 года 
- Посчитали количество обзоров и среднюю оценку
- Издательство, которое выпустило наибольшее число книг больше 50 страниц, - Penguin Books
- Самая высокая средняя оценка книг у J.K. Rowling/Mary GrandPré
- Среднее количество обзоров на пользователя, который поставил более 48 оценок, - 24.